In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


### Define the Class (Network)

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), # nn.conv != nn.Linear !!!!
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9], device='cuda:0')


In [8]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [10]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-3.3073e-01, -9.0877e-02,  8.7880e-02, -1.2368e-01,  2.3906e-01,
          6.2492e-01,  3.6156e-01,  4.2439e-01, -5.2242e-02,  2.0641e-01,
          2.1417e-01, -1.1319e-01,  4.2537e-01, -1.8973e-01, -6.9214e-01,
         -6.0857e-02, -2.1962e-01, -4.9146e-01,  2.3117e-01, -7.6042e-01],
        [ 1.0937e-01, -1.2490e-01, -7.0274e-02,  7.3951e-02, -1.9002e-02,
          3.5069e-01,  4.6073e-01,  4.5088e-01, -3.9649e-02,  2.5548e-01,
         -1.7075e-01, -1.3303e-01, -8.3693e-02, -6.8592e-04, -3.6841e-01,
         -9.3755e-02, -1.1012e-01, -3.3221e-01,  2.0922e-01, -3.2240e-01],
        [-2.0938e-01, -8.9202e-02, -2.3541e-01,  2.8254e-02, -8.2179e-02,
          5.5902e-01,  5.6596e-02,  5.2451e-01,  2.8117e-01,  9.8963e-02,
          4.0708e-02,  6.4708e-03,  2.5393e-01, -5.2896e-02, -3.7457e-01,
         -2.6072e-01, -2.6999e-01, -3.0803e-01,  9.8120e-02, -5.4672e-01]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0879, 0.0000, 0.2391,

In [17]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
logits

tensor([[-0.3432,  0.0341, -0.3010, -0.0560, -0.1086,  0.1440,  0.0617,  0.2010,
          0.1049,  0.0315],
        [-0.2447, -0.0276, -0.2343,  0.0067, -0.1324,  0.0948,  0.1304,  0.1822,
          0.0973,  0.1523],
        [-0.2133, -0.0281, -0.2186, -0.0427, -0.1702,  0.1490, -0.0048,  0.2015,
          0.0813,  0.1046]], grad_fn=<AddmmBackward0>)

In [16]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab

tensor([[0.0809, 0.1059, 0.1148, 0.0944, 0.0985, 0.1446, 0.0830, 0.0892, 0.0895,
         0.0993],
        [0.0732, 0.1048, 0.1127, 0.0892, 0.1062, 0.1495, 0.0871, 0.0890, 0.0903,
         0.0980],
        [0.0767, 0.1106, 0.1103, 0.0876, 0.0986, 0.1240, 0.0972, 0.0961, 0.0963,
         0.1025]], grad_fn=<SoftmaxBackward0>)

In [15]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0226,  0.0227,  0.0235,  ...,  0.0349, -0.0214,  0.0183],
        [ 0.0253, -0.0007, -0.0308,  ...,  0.0094,  0.0146,  0.0215]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0050, -0.0078], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0362,  0.0162, -0.0010,  ...,  0.0042, -0.0410,  0.0387],
        [-0.0187, -0.0302, -0.0357,  ..., -0.0027,  0.0287,  0.0227]],
       device='cuda:0', grad_fn=<Sl